# FAQ EXTRACTION

In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import wordpunct_tokenize
from nltk import pos_tag
import csv

## DATA CLEANSING

Reading CSVs

In [2]:
topics = pd.read_csv("C:/Users/hp/Desktop/topic_dataset.csv")
question_bank = pd.read_csv("C:/Users/hp/Desktop/500_questions.csv")

Dropped unessential columns

In [3]:
topics.drop('Unnamed: 0', axis = 1, inplace =True)
question_bank.drop('Unnamed: 0',axis = 1, inplace = True)

Converting all the strings to lowercase

In [4]:
x = 0
while x< topics.shape[0]:
    topics.iloc[x].topic = topics.iloc[x].topic.lower()
    topics.iloc[x].subject = topics.iloc[x].subject.lower()
    x = x+1
x = 0
while x<question_bank.shape[0]:
    question_bank.iloc[x].question = question_bank.iloc[x].question.lower()
    x = x+1

In [5]:
topics.head()

,topic,subject
0,fcfs,operating system
1,dml,database management system
2,jdbc,object_oriented_programing
3,joints,database management system
4,buffering,operating system


In [6]:
question_bank.head()

,question
0,(avg. of 42 ) : what is a sql view?
1,question involving primary key integrity const...
2,how secure are encrypted stored procedures tr...
3,8. a candidate appearing for an examinatio �n ...
4,consider the following areas for tuning. the o...


## DATA REFINING and WAREHOUSING

Algorithms like RAKE, TF-IDF do not perform well on short sentences

So instead of using these algorithms, metadata of the dataset was created

Creating and filling new fields with empty lists to fill with metadata later

In [7]:
question_bank['related_words']=[[] for k in question_bank.question]

In [8]:
question_bank['tags']=[[] for k in question_bank.question]

In [9]:
question_bank['subject']=[[] for k in question_bank.question]

Finding important words in a question(usually nouns,verbs and adjectives)using wordpunct_tokenize and pos_tag

In [10]:
x =0
while x<question_bank.shape[0]:
    words = wordpunct_tokenize(question_bank.question[x])
    temp = pos_tag(words)
    for abc in temp:
        if abc[1] in ['JJ','VBN','NN','NNS','VB','VBD','VBG'] and len(abc[0])>2:
            question_bank['related_words'][x].append(abc[0])
    question_bank['related_words'][x] = list(set(question_bank['related_words'][x]))
    x = x+1
question_bank.head()

,question,related_words,tags,subject
0,(avg. of 42 ) : what is a sql view?,"[avg, view, sql]",[],[]
1,question involving primary key integrity const...,"[constraint, question, primary, etc, involving...",[],[]
2,how secure are encrypted stored procedures tr...,"[secure, views, encrypted, triggers, stored, p...",[],[]
3,8. a candidate appearing for an examinatio �n ...,"[appearing, secure, candidate, failed, secured...",[],[]
4,consider the following areas for tuning. the o...,"[prevent, consider, reduce, order, maintained,...",[],[]


Tagging each question with a particular topic by similar string matching

In [11]:
x = 0
while x<question_bank.shape[0]:
    y = 0
    while y<topics.shape[0]:
        if(len(topics.topic[y])>=4):
            z = topics.topic[y][:int(len(topics.topic[y])*0.7)]
        else:
            z = topics.topic[y]
        if question_bank.question[x].find(z)!= -1:
            question_bank.tags[x].append(topics.topic[y])
        y = y+1
    x = x+1
question_bank.head()

,question,related_words,tags,subject
0,(avg. of 42 ) : what is a sql view?,"[avg, view, sql]",[sql],[]
1,question involving primary key integrity const...,"[constraint, question, primary, etc, involving...",[keys],[]
2,how secure are encrypted stored procedures tr...,"[secure, views, encrypted, triggers, stored, p...",[security],[]
3,8. a candidate appearing for an examinatio �n ...,"[appearing, secure, candidate, failed, secured...",[security],[]
4,consider the following areas for tuning. the o...,"[prevent, consider, reduce, order, maintained,...","[buffering, caching, sql]",[]


Finding subject(s) under which the question comes

In [12]:
x = 0
while x<question_bank.shape[0]:
    y =0
    while y<len(question_bank.tags[x]):
        z = 0
        while z< topics.shape[0]:
            if question_bank.tags[x][y] == topics.topic[z]:
                if topics.subject[z] not in question_bank.subject[x]:
                    question_bank.subject[x].append(topics.subject[z])
            z = z+1
        y = y+1
    x = x+1
question_bank.head()

,question,related_words,tags,subject
0,(avg. of 42 ) : what is a sql view?,"[avg, view, sql]",[sql],[database management system]
1,question involving primary key integrity const...,"[constraint, question, primary, etc, involving...",[keys],[database management system]
2,how secure are encrypted stored procedures tr...,"[secure, views, encrypted, triggers, stored, p...",[security],[operating system]
3,8. a candidate appearing for an examinatio �n ...,"[appearing, secure, candidate, failed, secured...",[security],[operating system]
4,consider the following areas for tuning. the o...,"[prevent, consider, reduce, order, maintained,...","[buffering, caching, sql]","[operating system, database management system]"


## SEARCHING

We take the input from user to search

Refined searching using related words

In [13]:
def relate_words_search(question_bank,res,search_word,additional,do_check):
    x = 0
    if len(search_word)>3:
        s = search_word[:int(len(search_word)*0.85)]
    else:
        s = search_word
    while x<question_bank.shape[0]:
        check = -1
        for t in question_bank.related_words[x]:
            if t.find(s) != -1:
                res.append(question_bank.question[x])
                check = check +1
        if check != -1 and do_check ==1:
            for t in question_bank.related_words[x]:
                additional.append(t)
        x = x+1
    return additional

Function to extract subject to which the user input keyword is related

In [14]:
def extract(question_bank,search_word):
    x = 0
    while x<question_bank.shape[0]:
        if search_word in question_bank.related_words[x]:
            return question_bank.subject[x]
        x = x+1

Searching for content related to the subject

In [15]:
def subject_related_search(question_bank,res,search_word):
    d =extract(question_bank,search_word)
    x = 0
    while x < question_bank.shape[0]:
        for f in question_bank.subject[x]:
            if f in d:
                res.append(question_bank.question[x])
        x=x+1

Using shuffle method to not get the same output everytime

In [16]:
from random import shuffle

Tag searching

In [17]:
def tag_search(question_bank,res):
        x = 0
        while x<question_bank.shape[0]:
            if search_word in question_bank.tags[x]:
                res.append(question_bank.question[x])
            x = x+1
        return res

Main search{Relating to metadata and searching}
AND output

In [23]:
e =0
while e<topics.shape[0]:
    search_word = topics.topic[e]
    write_title(topics.subject[e],topics.topic[e])
    res = []
    additional=[]
    res = tag_search(question_bank,res)
    Next = []
    if len(res)<10:
        subject_related_search(question_bank,Next,search_word)
    if len(res)+len(Next)<10:
        additional=relate_words_search(question_bank,res,search_word,additional,1)
        for o in additional:
            x = relate_words_search(question_bank,Next,o,additional,0)
    res =list(set(res))
    for v in res:
        write_q(v)
    Next = list(set(Next)-set(res))
    shuffle(Next)
    v =0
    while v<=10-len(res)+1 and v<len(Next):
        write_q(Next[v])
        v =v+1

    e = e+1

## CSV FILE HANDLING

In [22]:
# OUTPUT WISE COLUMN NAMES
with open('Test.csv','w') as csvfile:
    fw = csv.writer(csvfile)
    fw.writerow(['SUBJECT','TOPIC','QUESTION'])

In [19]:
def write_title(Subject,Topic):
    with open('Test.csv','a') as csvfile:
        fw = csv.writer(csvfile)
        fw.writerow([Subject,Topic,''])

In [20]:
def write_q(question):
    with open('Test.csv','a') as csvfile:
        fw = csv.writer(csvfile)
        fw.writerow(['','',question])